# Data Cleaning

**Date**: 5/11/2025

**Summary**: This file contains all data cleaning for this project. The structure is as follows:
1. Library Imports and Data Loading
2. Cleaning main_data
3. Cleaning survey_data
4. Merging main_data and survey_data with additional cleaning
5. Additionally merging with hosts_data with additional cleaning

## Library Imports and Data Loading

In [123]:
# Import libaries
import pandas as pd
import numpy as np

In [124]:
# Import datasets
main_data = pd.read_csv("raw_data/main_data.csv", header=None)
hosts_data = pd.read_stata("raw_data/hosts.dta")
survey_data = pd.read_excel("raw_data/name_survey_results.xlsx")

## Clean `main_data`

In [125]:
# Rename main_data columns
main_data.columns = [
    "host_response",
    "response_date",
    "number_of_messages",
    "automated_coding",
    "latitude",
    "longitude",
    "bed_type",
    "property_type",
    "cancellation_policy",
    "number_guests",
    "bedrooms",
    "bathrooms",
    "cleaning_fee",
    "price",
    "apt_rating",
    "property_setup",
    "city",
    "date_sent",
    "listing_down",
    "number_of_listings",
    "number_of_reviews",
    "member_since",
    "verified_id",
    "host_race",
    "super_host",
    "host_gender",
    "host_age",
    "host_gender_1",
    "host_gender_2",
    "host_gender_3",
    "host_race_1",
    "host_race_2",
    "host_race_3",
    "guest_first_name",
    "guest_last_name",
    "guest_race",
    "guest_gender",
    "guest_id",
    "population",
    "whites",
    "blacks",
    "asians",
    "hispanics",
    "available_september",
    "up_not_available_september",
    "september_price",
    "census_tract",
    "host_id",
    "new_number_of_listings",
]

In [126]:
# Convert all columns to strings for cleaning
for col in main_data.columns:
    main_data[col] = main_data[col].astype(str)

# Convert all missing values to "."
variables_to_clean = main_data.loc[:, "response_date":"september_price"].columns.tolist()
for col in variables_to_clean:
    main_data[col] = main_data[col].replace(["\\N", "Null", "-1"], ".")

In [127]:
# Convert numeric columns back into numeric form (from string)
columns_to_destring = [
    "host_response", 
    "number_of_messages", 
    "automated_coding", 
    "latitude", 
    "longitude",
    "number_guests", 
    "bedrooms", 
    "bathrooms", 
    "cleaning_fee", 
    "price", 
    "apt_rating", 
    "listing_down",
    "number_of_listings", 
    "number_of_reviews", 
    "verified_id", 
    "super_host", 
    "guest_id",
    "population", 
    "whites", 
    "blacks", 
    "hispanics", 
    "asians", 
    "available_september",
    "up_not_available_september", 
    "september_price", 
    "host_id", 
    "new_number_of_listings"
]

# Convert these columns to numeric
for col in columns_to_destring:
    main_data[col] = pd.to_numeric(main_data[col], errors="coerce")


In [128]:
# Convert missing values to NaN before converting to datetime
main_data['response_date'] = main_data['response_date'].replace('.', np.nan)
main_data['date_sent'] = main_data['date_sent'].replace('.', np.nan)

# Convert date columns to datetime format
main_data["response_date_stata"] = pd.to_datetime(
    main_data["response_date"],
    errors="coerce"
)
main_data["date_sent_stata"] = pd.to_datetime(
    main_data["date_sent"],
    errors="coerce"
)

In [129]:
# Create binary variables for race and gender
main_data["guest_black"] = (main_data["guest_race"].str.lower() == "black").astype(int)
main_data["guest_white"] = (main_data["guest_black"] == 0).astype(int)
main_data["guest_female"] = (main_data["guest_gender"].str.lower() == "female").astype(int)
main_data["guest_male"] = (main_data["guest_gender"].str.lower() == "male").astype(int)

In [130]:
# Create a combined guest_name * city variable for clustered standard errors
main_data["name_by_city"] = main_data["guest_first_name"] + main_data["city"]

In [131]:
# Sort by guest_first_name
main_data_sorted = main_data.sort_values(by="guest_first_name")

In [132]:
# Export for analysis
main_data_sorted.to_csv("clean_data/main_data_cleaned.csv", index=False)

## Clean `survey_data`

In [133]:
# Sort by guest_first_name
survey_data_sorted = survey_data.sort_values(by="guest_first_name")

In [134]:
# Export for analysis
survey_data_sorted.to_csv("clean_data/survey_data_cleaned.csv", index=False)

## Merging main_data and survey_data with additional cleaning

In [135]:
# Merge dataframes
merged_data = pd.merge(
    main_data_sorted, 
    survey_data_sorted, 
    on="guest_first_name", 
    how="left",
    indicator="survey_merge"
)

In [136]:
# Convert the indicator to match Stata's output
merged_data["survey_merge"] = merged_data["survey_merge"].map({
    "left_only": 1,  # Only in main dataset
    "right_only": 2, # Only in survey dataset
    "both": 3        # In both datasets
})

In [137]:
# Adjust continuous race score (0 to 1, not 1 to 2)
merged_data["guest_race_continuous"] = merged_data["guest_race_continuous"] - 1

In [138]:
# Create binary variables for host race
for race in ["black", "white", "hisp", "asian", "mult"]:
    column_name = f"host_race_{race}"
    merged_data[column_name] = 0
    merged_data.loc[merged_data["host_race"] == race, column_name] = 1

In [139]:
# Create binary variables for host gender
for gender in ["F", "FF", "M", "MM", "MF"]:
    merged_data[f"host_gender_{gender}"] = (merged_data["host_gender"] == gender).astype(int)

# Create host_gender_same_sex variable
merged_data["host_gender_same_sex"] = ((merged_data["host_gender_MM"] == 1) | 
                                       (merged_data["host_gender_FF"] == 1)).astype(int)

In [140]:
# Create categorical host age variable
merged_data["host_age_cat"] = np.nan

# Map different age groups to categorical values
young_ages = ["young", "young/UU", "UU/young", "young/NA", "NA/young"]
middle_young_ages = ["middle/young", "young/middle"]
middle_ages = ["middle", "middle/UU", "UU/middle", "middle/NA", "NA/middle"]
middle_old_ages = ["middle/old", "old/middle"]
old_ages = ["old", "old/UU", "UU/old", "old/NA", "NA/old"]

# Apply categorization
merged_data.loc[merged_data["host_age"].isin(young_ages), "host_age_cat"] = 0
merged_data.loc[merged_data["host_age"].isin(middle_young_ages), "host_age_cat"] = 1
merged_data.loc[merged_data["host_age"].isin(middle_ages), "host_age_cat"] = 2
merged_data.loc[merged_data["host_age"].isin(middle_old_ages), "host_age_cat"] = 3
merged_data.loc[merged_data["host_age"].isin(old_ages), "host_age_cat"] = 4

In [141]:
# Create binary variables for other host/property characteristics
merged_data["ten_reviews"] = (merged_data["number_of_reviews"] >= 10).astype(int)
merged_data["five_star_property"] = (merged_data["apt_rating"] == 5).astype(int)
merged_data["multiple_listings"] = (merged_data["number_of_listings"] > 1).astype(int)
merged_data["shared_property"] = ((merged_data["property_setup"] == "Private Room") | 
                                 (merged_data["property_setup"] == "Shared Room")).astype(int)
merged_data["shared_bathroom"] = ((merged_data["shared_property"] == 1) & 
                                 (merged_data["bathrooms"] < 1.5)).astype(int)
merged_data["has_cleaning_fee"] = (~merged_data["cleaning_fee"].isna()).astype(int)
merged_data["strict_cancellation"] = (merged_data["cancellation_policy"] == "Strict").astype(int)
merged_data["young"] = (merged_data["host_age_cat"] == 0).astype(int)
merged_data["middle"] = ((merged_data["host_age_cat"] == 1) | 
                        (merged_data["host_age_cat"] == 2)).astype(int)
merged_data["old"] = ((merged_data["host_age_cat"] == 3) | 
                     (merged_data["host_age_cat"] == 4)).astype(int)


In [142]:
# Replace all nan and .'s with np.nan
merged_data = merged_data.replace(["nan", "."], np.NaN)

In [143]:
# Calculate price statistics
top_decile_price = merged_data["price"].quantile(0.9)
median_price = merged_data["price"].median()

merged_data["pricey"] = (merged_data["price"] >= top_decile_price).astype(int)
merged_data["price_median"] = (merged_data["price"] > median_price).astype(int)
merged_data["log_price"] = np.log(merged_data["price"])

In [144]:
# Calculate population proportions
merged_data["white_proportion"] = merged_data["whites"] / merged_data["population"]
merged_data["black_proportion"] = merged_data["blacks"] / merged_data["population"]
merged_data["asian_proportion"] = merged_data["asians"] / merged_data["population"]
merged_data["hispanic_proportion"] = merged_data["hispanics"] / merged_data["population"]

In [145]:
# Calculate listings per census tract (group by census tract and count listings)
tract_counts = merged_data.groupby("census_tract")["latitude"].count().reset_index()
tract_counts.columns = ["census_tract", "tract_listings"]

# Merge back to main dataframe
merged_data = pd.merge(merged_data, tract_counts, on="census_tract", how="left")
merged_data["log_tract_listings"] = np.log(merged_data["tract_listings"])

/Users/mayaumlauf/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [146]:
# Create labeled host response variables
host_response_labels = {
    0: "No or unavailable",
    1: "Yes",
    2: "Request for more info (Can you verify? How many people?)",
    3: "No, unless you verify",
    4: "Yes, if you verify/give more info",
    5: "Offers a different place",
    6: "Offers Lower Price If You Book Now",
    7: "Asks for higher price",
    8: "Yes if stay is extended",
    9: "Check back later for definitive answer",
    10: "I will get back to you",
    11: "Unsure right now",
    12: "Only used for events",
    13: "Confused (our date error)",
    14: "Message not sent",
    -1: "No response"
}

# Create simplified host response
merged_data["simplified_host_response"] = np.nan

# Map various host responses to simplified categories
response_mapping = {
    1: 1,  # Yes
    4: 2,  # Yes, but requests more info
    6: 3,  # Yes, with lower price if booked now
    8: 4,  # Yes, if guest extends stay
    5: 5,  # Yes, but in different property
    7: 6,  # Yes, at a higher price
    2: 7,  # Requests more information
    -1: 9, # No response
    3: 10, # No, without more information
    0: 11  # No
}

# Apply the mapping for values 0-11
for original, simplified in response_mapping.items():
    merged_data.loc[merged_data["host_response"] == original, "simplified_host_response"] = simplified

# Set responses 9-11 to category 8
merged_data.loc[(merged_data["host_response"] >= 9) & 
               (merged_data["host_response"] <= 11), "simplified_host_response"] = 8

# Set responses 12-14 to NaN
merged_data.loc[(merged_data["host_response"] >= 12) & 
               (merged_data["host_response"] <= 14), "simplified_host_response"] = np.nan

# Create even more simplified response variable for Figure 2
merged_data["graph_bins"] = np.nan

# Map simplified responses to graph bins
merged_data.loc[merged_data["simplified_host_response"] == 1, "graph_bins"] = 1  # Yes
merged_data.loc[(merged_data["simplified_host_response"] >= 2) & 
               (merged_data["simplified_host_response"] <= 6), "graph_bins"] = 2  # Conditional Yes
merged_data.loc[merged_data["simplified_host_response"] == 9, "graph_bins"] = 3  # No Response
merged_data.loc[(merged_data["simplified_host_response"] == 7) | 
               (merged_data["simplified_host_response"] == 8) | 
               (merged_data["simplified_host_response"] == 10), "graph_bins"] = 4  # Conditional No
merged_data.loc[merged_data["simplified_host_response"] == 11, "graph_bins"] = 5  # No

# Create binary yes/no response variable
merged_data["yes"] = np.nan
merged_data.loc[(merged_data["host_response"] == 1) | 
               (merged_data["host_response"] == 4) | 
               (merged_data["host_response"] == 6), "yes"] = 1  # Yes responses
               
merged_data.loc[(merged_data["host_response"] == 0) | 
               (merged_data["host_response"] == -1) | 
               (merged_data["host_response"] == 2) | 
               (merged_data["host_response"] == 3) | 
               ((merged_data["host_response"] > 6) & 
                (merged_data["host_response"] < 13)), "yes"] = 0  # No responses

In [147]:
# Remove Tampa and Atlanta
merged_data = merged_data[~merged_data["city"].isin(["Tampa", "Atlanta"])]

In [148]:
# Create city indicators
merged_data["baltimore"] = (merged_data["city"] == "Baltimore").astype(int)
merged_data["dallas"] = (merged_data["city"] == "Dallas").astype(int)
merged_data["los_angeles"] = (merged_data["city"] == "Los-Angeles").astype(int)
merged_data["sl"] = (merged_data["city"] == "St-Louis").astype(int)
merged_data["dc"] = (merged_data["city"] == "Washington").astype(int)

In [149]:
# Sort by host
merged_data = merged_data.sort_values(by="host_id")

# Export for analysis (merged/clean main_data and survey_data)
merged_data.to_csv("clean_data/merged_main_survey_data.csv", index=False)

## Additionally merging with `hosts_data` with additional cleaning

In [150]:
# Merge with hosts data
merged_with_hosts = pd.merge(merged_data, hosts_data, on="host_id", how="left", indicator=True)
merged_with_hosts = merged_with_hosts[merged_with_hosts["_merge"] != "right_only"]
merged_with_hosts = merged_with_hosts.drop(columns=["_merge"])

In [151]:
# Calculate probability of being filled in September
merged_with_hosts["filled_september"] = (merged_with_hosts["up_not_available_september"] == 1).astype(int)

In [152]:
# Export for analysis (merged/clean main_data, survey_data, and hosts_data)
merged_with_hosts.to_csv("clean_data/merged_with_hosts.csv", index=False)